In [ ]:
# Databricks notebook source
# model_train.py
import mlflow
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from mlflow.models.signature import infer_signature # मॉडल सिग्नेचर के लिए
import os
import sys
import warnings

# चेतावनी (Warnings) को अनदेखा करें
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# ==================== CONFIGURATION ====================
EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Delta_RF"
MODEL_ARTIFACT_PATH = "sklearn_rf_model"
DELTA_TABLE_NAME = "house_price_delta" 

# मॉडल और डेटा कॉन्फ़िगरेशन
RANDOM_STATE = 42
TEST_SIZE = 0.2
FEATURE_COLS = ['sq_feet', 'num_bedrooms', 'num_bathrooms', 'year_built', 'location_score']
LABEL_COL = 'price'

# Cross-validation के लिए parameter grid
PARAM_GRID = {
    'n_estimators': [100, 150, 200],
    'max_depth': [10, 12, 15],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
CV_FOLDS = 3

# ==================== FUNCTIONS ====================

def setup_mlflow_experiment():
    """MLflow ट्रैकिंग और रजिस्ट्री को Databricks UC के लिए कॉन्फ़िगर करता है।"""
    if "DATABRICKS_RUNTIME_VERSION" in os.environ:
        try:
            mlflow.set_tracking_uri("databricks") 
            # UC रजिस्ट्रेशन के लिए यह आवश्यक है
            mlflow.set_registry_uri("databricks-uc") 
            print("✓ MLflow configured for Databricks UC (Tracking & Registry).")
        except Exception as e:
            # यदि कॉन्फ़िगरेशन विफल होता है, तो आगे बढ़ें लेकिन चेतावनी दें
            print(f"⚠ Warning: MLflow Registry setup failed with: {e}")
            
    try:
        mlflow.set_experiment(EXPERIMENT_NAME)
        print(f"✓ MLflow Experiment set to: {EXPERIMENT_NAME}")
    except Exception as e:
        print(f"❌ Critical: MLflow Experiment setup failed! Error: {e}")
        pass


def get_data_for_training(spark: SparkSession, table_name: str):
    """डेल्टा टेबल से डेटा लोड करता है और Pandas DataFrame में बदलता है।"""
    print(f"💾 Loading data from Delta Table: {table_name}")
    try:
        # Delta Table से Spark DataFrame लोड करें
        df_spark = spark.read.format("delta").table(table_name)
        
        # आवश्यक कॉलम चुनें और Pandas DataFrame में बदलें
        df_pd = df_spark.select(*FEATURE_COLS, col(LABEL_COL)).toPandas()
        
        print(f"✓ Data loaded. Total rows: {len(df_pd)}")
        
        X = df_pd[FEATURE_COLS]
        y = df_pd[LABEL_COL]
        
        return X, y
        
    except Exception as e:
        print(f"❌ Error loading data from Delta: {e}")
        return None, None


def train_and_log_model(X, y):
    """
    Cross-validation के साथ मॉडल को ट्रेन करता है, best parameters ढूंढता है,
    मीट्रिक्स की गणना करता है और UC रजिस्ट्रेशन के लिए आवश्यक मॉडल सिग्नेचर के साथ MLflow में लॉग करता है।
    """
    
    # 1. डेटा स्प्लिट करें
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )
    
    with mlflow.start_run(run_name="RandomForest_House_Price_Model_CV") as run:
        run_id = run.info.run_id
        print(f"🚀 MLflow Run Started with ID: {run_id}")
        
        # 2. Cross-validation setup
        print(f"🔍 Starting GridSearchCV with {CV_FOLDS}-fold cross-validation...")
        print(f"📊 Parameter Grid: {PARAM_GRID}")
        
        base_model = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1)
        
        grid_search = GridSearchCV(
            estimator=base_model,
            param_grid=PARAM_GRID,
            cv=CV_FOLDS,
            scoring='neg_mean_squared_error',
            n_jobs=-1,
            verbose=1
        )
        
        # 3. मॉडल ट्रेनिंग with cross-validation
        grid_search.fit(X_train, y_train)
        print("✓ GridSearchCV completed successfully.")
        
        # 4. Best parameters को लॉग करें
        best_params = grid_search.best_params_
        print(f"✓ Best Parameters found: {best_params}")
        
        for param_name, param_value in best_params.items():
            mlflow.log_param(f"best_{param_name}", param_value)
        
        # CV score को भी लॉग करें
        best_cv_score = -grid_search.best_score_  # negative से positive में बदलें
        mlflow.log_metric("best_cv_rmse", np.sqrt(best_cv_score))
        print(f"✓ Best CV RMSE: {np.sqrt(best_cv_score):.2f}")
        
        # 5. Best model के साथ test set पर evaluate करें
        best_model = grid_search.best_estimator_
        predictions = best_model.predict(X_test)
        
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        r2 = r2_score(y_test, predictions)

        mlflow.log_metric("test_rmse", rmse)
        mlflow.log_metric("test_r2_score", r2)
        print(f"✓ Test Metrics Logged: RMSE={rmse:.2f}, R2={r2:.4f}")
        
        # 6. मॉडल सिग्नेचर बनाएं (UC के लिए अनिवार्य)
        model_signature = infer_signature(
            X_train,
            best_model.predict(X_train)
        )
        print("✓ Model Signature created successfully.")

        # 7. Best model को लॉग करें (सिग्नेचर सहित)
        mlflow.sklearn.log_model(
            sk_model=best_model, 
            artifact_path=MODEL_ARTIFACT_PATH,
            signature=model_signature,
            registered_model_name=None 
        )
        print(f"✓ Best model logged with signature to artifact path: {MODEL_ARTIFACT_PATH}")
        
        return run_id

# ==================== EXECUTION ====================

if __name__ == "__main__":
    try:
        # SparkSession को इनिशियलाइज़ या प्राप्त करें
        spark = SparkSession.builder.appName("ModelTrain").getOrCreate()
    except Exception as e:
        print(f"❌ SparkSession creation failed: {e}")
        sys.exit(1)

    setup_mlflow_experiment()
    
    # डेटा लोड करें
    X, y = get_data_for_training(spark, DELTA_TABLE_NAME)

    if X is not None:
        # मॉडल ट्रेन और लॉग करें
        training_run_id = train_and_log_model(X, y)
        
        if training_run_id:
            print("\n" + "=" * 60)
            print(f"✅ TRAINING & LOGGING COMPLETE! New Run ID: {training_run_id}")
            print(f"अगला कदम: 'model_register.py' को इस Run ID के साथ चलाएँ।")
            print("=" * 60)
        else:
            sys.exit(1)
    else:
        sys.exit(1)